1. Imports y configuración basica

In [1]:
import os
import pyreadr
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


# For progress
from tqdm.auto import tqdm


tqdm.pandas()


# %%
# Paths (adjust if needed)
DATA_DIR = os.path.join(os.getcwd(), 'DatasetReducido')
OUTPUT_DIR = os.path.join(os.getcwd(), 'processed')
os.makedirs(OUTPUT_DIR, exist_ok=True)


# Reduced filenames (expectation)
reduced_files = {
'FaultFree_Training': os.path.join(DATA_DIR, 'FaultFree_Training_reduced.RData'),
'FaultFree_Testing': os.path.join(DATA_DIR, 'FaultFree_Testing_reduced.RData'),
'Faulty_Training': os.path.join(DATA_DIR, 'Faulty_Training_reduced.RData'),
'Faulty_Testing': os.path.join(DATA_DIR, 'Faulty_Testing_reduced.RData')
}


print('Using DATA_DIR =', DATA_DIR)
print('Output will be saved to =', OUTPUT_DIR)

Using DATA_DIR = C:\Users\Oihane\Desktop\Analitica de datos\proyecto_analitica\DatasetReducido
Output will be saved to = C:\Users\Oihane\Desktop\Analitica de datos\proyecto_analitica\processed


In [3]:
def load_first_df_from_rdata(path):
    """Load first object from RData using pyreadr.
    Returns a pandas DataFrame or raises FileNotFoundError/ValueError."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    res = pyreadr.read_r(path)
    if len(res.keys()) == 0:
        raise ValueError(f'No objects found in {path}')
    key = list(res.keys())[0]
    df = res[key]
    # Ensure proper dtypes for 'sample' and 'simulationRun'
    if 'sample' in df.columns:
        df['sample'] = pd.to_numeric(df['sample'], errors='coerce').astype('Int64')
    if 'simulationRun' in df.columns:
        df['simulationRun'] = pd.to_numeric(df['simulationRun'], errors='coerce').astype('Int64')
    return df

In [4]:
datasets = {}
for name, path in reduced_files.items():
    try:
        datasets[name] = load_first_df_from_rdata(path)
        print(f'Loaded {name}:', datasets[name].shape)
    except Exception as e:
        print(f'Warning: could not load {name} from {path}: {e}')
        datasets[name] = pd.DataFrame()